📘Задание 1. Обязательная часть Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests

In [2]:
# Посылаю запрос к сайту, используя метод get
URL = 'https://habr.com/ru/all/'
req = requests.get(URL)
req.text

'<!DOCTYPE html>\n<html lang="ru" class="no-js">\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<meta content=\'width=1024\' name=\'viewport\'>\n<title>Все публикации подряд / Хабр</title>\n\n\n\n  <meta property="fb:app_id" content="444736788986613" />\n<meta property="og:type" content="website"/>\n<meta property="fb:pages" content="472597926099084"/>\n<meta property="og:site_name" content="Хабр" />\n<link rel="image_src" href="https://habrastorage.org/storage/stuff/habr/habr_ru.png" />\n<meta property="og:image" content="https://habrastorage.org/storage/stuff/habr/habr_ru.png" />\n<meta property="og:image:width" content="1200" />\n<meta property="og:image:height" content="628" />\n<meta property="og:title" content="Все публикации подряд / Хабр"/>\n<meta property="og:description" content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом."/>\n\n\n\n<meta name=\'yandex-verification\

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(req.text)
type(soup)

bs4.BeautifulSoup

In [5]:
# Извлекаю посты с главной страницы
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/hetmansoftware/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/8d6/0c8/95b/8d60c895b23ce4ad77e0544574068c80.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">hetmansoftware</span>
 </a>
 <span class="post__time">сегодня в 22:58</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/hetmansoftware/blog/549404/">Лучшие флаги «Google Chrome» для повышения удобства пользователей</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/company/hetmansoftware/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Блог компании Hetm

In [6]:
import pandas as pd

In [7]:
import time

In [8]:
import datetime

In [9]:
#Добавила с список искомые слова с большой буквы
# Ищу и записываю данные в датафрейм
KEYWORDS = ['python', 'парсинг']
Python = KEYWORDS[0].title()
Parsing = KEYWORDS[1].title()
KEYWORDS_WITH_TITLE = KEYWORDS+[Python,Parsing]
total = pd.DataFrame()
for post in posts:
    for i in KEYWORDS_WITH_TITLE:
        if i in post.text:
            time.sleep(0.3)
            post_preview = post.find('a', class_='post__title_link')
            data = post.find('span', class_='post__time').text
            if 'сегодня' in data:
                date = pd.to_datetime('today').date()
            elif 'вчера' in data:
                date = pd.to_datetime('today').date() - timedelta(days = 1)
            link_ = post_preview.get('href')
            title_text = post_preview.text
            row = {'date': date.strftime('%Y/%m/%d'), 'title': title_text, 'link': link_}
            total = pd.concat([total, pd.DataFrame([row])])  
total.reset_index()

,index,date,title,link
0,0,2021/03/28,Перенос данных из VisionFlow в ServiceNow,https://habr.com/ru/post/549400/
1,0,2021/03/28,Анализ и построение ROC-кривых: связь с РЛС,https://habr.com/ru/post/549376/


📘Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [40]:
# Отправляю на проверку список адресов на сайт, используя метод post
EMAIL = ['xxx@x.ru', 'yyy@y.com', 'zzz@z.ru']
#EMAIL = ['xxx@x.ru', 'yyy@y.com']
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
'Vaar-Header-App-Build-Version': '1.0.0',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
'Vaar-Version': '0'
}

post_req = requests.post(url, json = {'emailAddresses': EMAIL}, headers=headers)
   

In [41]:
#Получаю словарь с данными
dict_response = post_req.json()
dict_response

{'breaches': {'16613': {'breachId': 16613,
   'site': 'verifications.io',
   'recordsCount': 677914246,
   'description': 'Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked database contained information such as physical addresses, phone numbers, email addresses, dates of birth, gender, employer and job information, geographic location and IP addresses.\r\n\r\n',
   'publishDate': '2019-03-28T00:00:00Z',
   'statistics': {'usernames': 0, 'passwords': 0, 'emails': 677914246}},
  '17110': {'breachId': 17110,
   'site': 'azcentral.com',
   'recordsCount': 705538,
   'description': 'At an unconfirmed date, online Arizona newspaper "AZ Central" was allegedly breached. The stolen data contains passwords and email ad

In [37]:
# Созадала словарь с ключами - е-майлами, которые мы проверяем, и значениями - списками, источниками слива информации
def create_dictionary_address_source(emails):
    """
    функция создает словарь из полученных данных в формате {запрашиваемый адрес: список адресов,источников слива информации}
    """
    list_source = []
    dict_emails = {}
    for i in range(len(emails)):
        list_source.append([])
        for keys, values in dict_response['data'].items():
            for elem in values:
                if elem == emails[i]:
                    list_source[i].append(keys)
    for e,f in enumerate(emails):
            dict_emails[f] = list_source[e]
    return dict_emails

dict_emails = create_dictionary_address_source(EMAIL)
print(dict_emails)


{'xxx@x.ru': ['parapa.mail.ru', 'adobe.com', 'cdprojektred.com', 'cfire.mail.ru', 'vk.com', 'imesh.com'], 'yyy@y.com': ['adobe.com', 'azcentral.com', 'linkedin.com', 'dropbox.com', 'wishbone.io', 'youku.com', 'myheritage.com', 'rayli.com.cn', 'canva.com', 'globalreach.eu', 'forums.vkmonline.com', 'netlog.com', 'imesh.com', 'zynga.com'], 'zzz@z.ru': ['verifications.io', 'adobe.com', 'edaboard.com', 'cfire.mail.ru', 'vk.com']}


In [38]:
import pandas as pd

In [39]:
#Создадаю датафрейм <почта> - <дата утечки> - <источник утечки> - <описание утечки>
df =pd.DataFrame()
for keys, values in dict_emails.items():
    for i in values:
            num_breach = dict_response['data'][i][keys][0]['breachId']
            date = pd.to_datetime(dict_response['breaches'][str(num_breach)]['publishDate'], dayfirst=True).date()
            row = {'email': keys,'date': date, 'source': i, 'description': dict_response['breaches'][str(num_breach)]['description']}
            df = pd.concat([df, pd.DataFrame([row])]) 
df.sort_values(['email','date']).reset_index().drop('index', 1) 

,email,date,source,description
0,xxx@x.ru,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
1,xxx@x.ru,2016-10-23,imesh.com,"In June 2016, a cache of over 51 million user ..."
2,xxx@x.ru,2016-10-29,vk.com,Popular Russian social networking platform VKo...
3,xxx@x.ru,2017-01-31,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
4,xxx@x.ru,2017-02-14,parapa.mail.ru,"In July and August 2016, two criminals execute..."
5,xxx@x.ru,2017-02-14,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
6,yyy@y.com,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
7,yyy@y.com,2016-10-21,linkedin.com,"In 2012, online professional networking platfo..."
8,yyy@y.com,2016-10-23,imesh.com,"In June 2016, a cache of over 51 million user ..."
9,yyy@y.com,2016-10-24,dropbox.com,Cloud storage company Dropbox suffered a major...
